# Understanding Weight sharing
Same set of weights (i.e. CNN kernal or filter) to be used by every stride. Here we understand how to reuse the same set of weights (Variable/tensor).

In [1]:
import numpy as np
import tensorflow as tf

# Create set of common weights (a CNN filter)

In [2]:
tf.reset_default_graph()

In [3]:
initial_weights = np.array([[1,2,3],[4,5,6],[7,8,9]])
initial_weights

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [4]:
cnn_filter1 = tf.Variable(initial_weights, dtype=tf.float32, name="cnn_filter1")
cnn_filter1

<tf.Variable 'cnn_filter1:0' shape=(3, 3) dtype=float32_ref>

In [5]:
cnn_filter1.name

'cnn_filter1:0'

# setup input image that needs to be <font color="red"> morphed </font> into target image

In [6]:
ip_image=np.array([[10,20,30,40,50,60],[70,80,90,100,110,120],[130,140,150,160,170,180.]])
ip_image

array([[ 10.,  20.,  30.,  40.,  50.,  60.],
       [ 70.,  80.,  90., 100., 110., 120.],
       [130., 140., 150., 160., 170., 180.]])

In [7]:
np.random.seed(123)
target_image = 2*ip_image + np.random.randint(0,5, size=[3,6])
target_image

array([[ 22.,  44.,  62.,  81., 103., 122.],
       [143., 161., 181., 200., 221., 241.],
       [260., 280., 301., 323., 344., 360.]])

### feed your input & target image images into DAG

In [8]:
im_tensor1 = tf.placeholder(dtype=tf.float32, name="im_tensor1")
im_tensor2 = tf.placeholder(dtype=tf.float32, name="im_tensor2")

# Share the weights or filter1 among two patches of image

In [9]:
# Create images patches
patch1 = im_tensor1[:, :3]     # This slicing is actually called strided_slice in tensorflow
patch2 = im_tensor1[:, 3:]

In [10]:
patch1.name,  patch2.name   

('strided_slice:0', 'strided_slice_1:0')

In [11]:
# Share the wieghts now 
activation_map1 = patch1*cnn_filter1      # Element-wise multiplication on tensors
activation_map2 = patch2*cnn_filter1

# Losses & Cost

In [12]:
loss1 = tf.square( im_tensor2[:, :3] - activation_map1, name="loss1")
loss2 = tf.square( im_tensor2[:, 3:] - activation_map2, name="loss2")

In [13]:
cost = tf.add( tf.sqrt(tf.reduce_sum(loss1, name="loss1_reduce_sum")), 
               tf.sqrt(tf.reduce_sum(loss2, name="loss2_reduce_sum")), name="cost")

# Optimizer

In [14]:
optimize = tf.train.GradientDescentOptimizer(learning_rate=0.001)
set_to_optimize = optimize.minimize(cost)

# Go ahead and execute

In [15]:
# Graph writer first
writer = tf.summary.FileWriter(logdir="./Graph", graph=tf.get_default_graph())

In [16]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### Just one pass of optimization at a time

In [17]:
# check starting cost
sess.run(cost, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})   

3467.4995

In [78]:
# 1-step optimize at a time and check revised cost (Observe how training happens)
sess.run(set_to_optimize, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})
sess.run(cost, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})

52.44408

In [79]:
# After finishing optimization (multiple runs of above cell), check CNN-Filter
sess.run(cnn_filter1)

array([[1.751467 , 2.0864246, 2.0581255],
       [2.0164278, 2.0104218, 2.0094392],
       [2.0140328, 1.9270368, 2.1294146]], dtype=float32)

In [80]:
# Remember the start weights?
initial_weights

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [ ]:
# Conclusion:
# All weights came close to 2. This is expected, as morphed image is just twice of original image

In [ ]:
sess.close()
writer.close()